<a href="https://colab.research.google.com/github/Frank-Dg/Paper-Information-Parser/blob/main/parser_paper_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

# 设置目标URL
url = 'https://proceedings.neurips.cc/paper_files/paper/2022'

# 发送HTTP GET请求
response = requests.get(url)
response.raise_for_status()  # 确保请求成功

# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(response.text, 'html.parser')

# 假设论文标题是在一些特定的HTML标签内，你需要根据实际的HTML结构来调整选择器
# 例如，如果论文标题在<h1>或<h2>标签内，可以使用soup.find_all('h1')或soup.find_all('h2')
papers_soup = soup.find_all('li', class_='conference')
titles=[]
urls=[]
authors=[]
for paper_soup in papers_soup:
    # 获取论文标题，它在<a>标签的title属性中
    title = paper_soup.find('a').text
    # 获取论文的URL路径，它在<a>标签的href属性中
    href = paper_soup.find('a').get('href')
    # 获取作者信息，它在<i>标签内
    author = paper_soup.find('i').text

    # print(f"Title: {title}")
    titles.append(title)
    #print(f"URL: https://proceedings.neurips.cc{href}")
    #print('https://proceedings.neurips.cc'+href)
    urls.append('https://proceedings.neurips.cc'+href)
    # print(f"Authors: {authors}\n")
    authors.append(author)
data = {
    'Title': titles,
    'URL': urls,
    'Authors': authors
}

import pandas as pd
df = pd.DataFrame(data)

# 导出到Excel文件
excel_file = 'NeurIPS2022_Papers.xlsx'
df.to_excel(excel_file, index=False, engine='openpyxl')

print(f'论文信息已保存到 {excel_file}')

论文信息已保存到 NeurIPS2022_Papers.xlsx


In [ ]:


# 假设urls是包含论文详细页面链接的列表
urls = df['URL'].tolist()  # 如果你已经有了DataFrame

# 用于存储论文标题和摘要的列表
papers_info = []


for url in urls:
    # 发送请求
    response = requests.get(url)
    # 如果响应状态码为200，即请求成功
    if response.status_code == 200:
        # 解析HTML内容
        soup = BeautifulSoup(response.content, 'html.parser')
        # 找到标题和摘要
        # 这里假设标题是在第一个<h4>标签，摘要是在跟随"Abstract"的第一个<p>标签里
        title = soup.find('h4').text.strip()
        abstract_tag = soup.find('h4', text='Abstract').find_next('p')
        abstract = abstract_tag.text.strip() if abstract_tag else 'No abstract available'
        # 将提取的信息添加到列表中
        #print(abstract)
        papers_info.append({'Title': title, 'Abstract': abstract})
    else:
        print(f"Failed to fetch {url}")

# 将结果转换为DataFrame
df_papers_info = pd.DataFrame(papers_info)
# 显示前几项数据以检查
print(df_papers_info.head())

# 保存到Excel文件
df_papers_info.to_excel("NeurIPS2022_Papers_Abstracts.xlsx", index=False)

<ipython-input-13-6c888ca04f43>:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  abstract_tag = soup.find('h4', text='Abstract').find_next('p')


                                               Title  \
0  Federated Submodel Optimization for Hot and Co...   
1  On Kernelized Multi-Armed Bandits with Constra...   
2       Geometric Order Learning for Rank Estimation   
3  Structured Recognition for Generative Models w...   
4  Fast Bayesian Coresets via Subsampling and Qua...   

                                            Abstract  
0  We focus on federated learning in practical re...  
1  We study a stochastic bandit problem with a ge...  
2  A novel approach to rank estimation, called ge...  
3  A key goal of unsupervised learning is to go b...  
4  Bayesian coresets approximate a posterior dist...  
